In [29]:
#!/usr/bin/env python3
import sympy as sp
from IPython.display import display, Math

# ---------- symbols ----------
q1, q2, q3 = sp.symbols('q1 q2 q3', real=True)          # q1=waist yaw, q2=hip pitch, q3=knee pitch
L1, L2, L3 = sp.symbols('L1 L2 L3', positive=True, real=True)
hx, hy, hz = sp.symbols('hx hy hz', real=True)          # hip offset (base->hip)

# ---------- homogeneous transforms ----------
def Tx(a):
    return sp.Matrix([[1,0,0,a],
                      [0,1,0,0],
                      [0,0,1,0],
                      [0,0,0,1]])

def Ty(a):
    return sp.Matrix([[1,0,0,0],
                      [0,1,0,a],
                      [0,0,1,0],
                      [0,0,0,1]])

def Tz(a):
    return sp.Matrix([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,1,a],
                      [0,0,0,1]])

def Rx(th):
    c, s = sp.cos(th), sp.sin(th)
    return sp.Matrix([[1,0,0,0],
                      [0,c,-s,0],
                      [0,s, c,0],
                      [0,0,0,1]])

def Ry(th):
    c, s = sp.cos(th), sp.sin(th)
    return sp.Matrix([[ c,0, s,0],
                      [ 0,1, 0,0],
                      [-s,0, c,0],
                      [ 0,0, 0,1]])

def Rz(th):
    c, s = sp.cos(th), sp.sin(th)
    return sp.Matrix([[c,-s,0,0],
                      [s, c,0,0],
                      [0, 0,1,0],
                      [0, 0,0,1]])

# ---------- RR leg chain ----------
# Base -> Hip offset
TBH = Tx(hx) * Ty(hy) * Tz(hz)

T_align = Ry(sp.pi/2)

# Hip yaw (waist) about Z
TH0 = Rz(q1)

# Hip pitch about Y, then link L1 along X
T01 = Ry(q2) * Tx(L1)

# Knee pitch about Y, then link (L2+L3) along X to foot/wheel-center
T12 = Ry(q3) * Tx(L2 + L3)

# Full transform: base -> foot
TBF = sp.simplify(TBH * T_align * TH0 * T01 * T12)
# TBF = sp.simplify(TBH * TH0 * T01 * T12)

# ---------- position ----------
x, y, z = sp.simplify(TBF[0,3]), sp.simplify(TBF[1,3]), sp.simplify(TBF[2,3])
p = sp.Matrix([x, y, z])
display(Math(r"[x,y,z]^T = " + sp.latex(p)))

# ---------- Jacobian (3x3) ----------
J = sp.simplify(p.jacobian([q1, q2, q3]))
display(Math(r"\mathbf{J}=" + sp.latex(J)))

# ---------- orientation (optional) ----------
R = TBF[:3,:3]
roll  = sp.atan2(R[2,1], R[2,2])
pitch = sp.atan2(-R[2,0], sp.sqrt(R[2,1]**2 + R[2,2]**2))
yaw   = sp.atan2(R[1,0], R[0,0])
pose_expr = sp.Matrix([x, y, z, roll, pitch, yaw])
# display(Math(r"[x,y,z,\phi,\theta,\psi]^T=" + sp.latex(pose_expr)))

# ---------- test substitution ----------
subs_test = {
    q1: sp.rad(0),
    q2: sp.rad(-90),
    q3: sp.rad(-140),
    L1: 0.2,
    L2: 0.24,
    L3: 0.0,
    hx: 0.0,
    hy: 0.2,   # RR hip offset (example)
    hz: 0.0
}
p_eval = p.subs(subs_test).evalf()
print("p_eval =", p_eval)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

p_eval = Matrix([[0.0161493336514453], [0.200000000000000], [0.154269026324769]])


In [30]:
#!/usr/bin/env python3
import sympy as sp

# symbols
xd, yd, zd = sp.symbols('xd yd zd', real=True)
L1, L2, L3 = sp.symbols('L1 L2 L3', positive=True, real=True)
hx, hy, hz = sp.symbols('hx hy hz', real=True)

L23 = L2 + L3

# --- inverse of align ---
# T_align = Ry(pi/2)  => R_align = Ry(pi/2)
# so to bring point into "aligned frame": pA = R_align.T * (p_des - h)
c, s = sp.cos(sp.pi/2), sp.sin(sp.pi/2)
R_align = sp.Matrix([[ c,0, s],
                     [ 0,1, 0],
                     [-s,0, c]])
p_des = sp.Matrix([xd, yd, zd])
h = sp.Matrix([hx, hy, hz])
pA = sp.simplify(R_align.T * (p_des - h))   # point in aligned frame

xA, yA, zA = pA[0], pA[1], pA[2]

# --- yaw about Z in aligned frame ---
q1_ik = sp.atan2(yA, -xA)

# --- planar 2-link in (r, zA) with links L1 and L23 ---
r = sp.sqrt(xA**2 + yA**2)
r2 = r**2 + zA**2

c3 = (r2 - L1**2 - L23**2) / (2*L1*L23)
# choose branch (elbow-up/down) ให้ตรงกับที่คุณนิยาม q3
s3 = -sp.sqrt(1 - c3**2)        # เปลี่ยนเป็น +sqrt(...) ถ้าต้องการอีก branch
q3_ik = sp.atan2(s3, c3)

q2_ik = -(sp.atan2(zA, r) - sp.atan2(L23*s3, L1 + L23*c3))

# degrees (optional)
rad2deg = 180/sp.pi
q1_deg = sp.simplify(q1_ik*rad2deg)
q2_deg = sp.simplify(q2_ik*rad2deg)
q3_deg = sp.simplify(q3_ik*rad2deg)

display(Math(r"q_1^{\circ} = " + sp.latex(q1_ik)))
display(Math(r"q_2^{\circ} = " + sp.latex(q2_ik)))
display(Math(r"q_3^{\circ} = " + sp.latex(q3_ik)))


test = {
    xd: 0.02,
    yd: 0.2,
    zd: 0.154,
    L1: 0.2,
    L2: 0.24,
    L3: 0.0,
    hx: 0.0,
    hy: 0.2,
    hz: 0.0
}

display(q1_deg.subs(test).evalf())
display(q2_deg.subs(test).evalf())
display(q3_deg.subs(test).evalf())

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

0

-91.3782874839520

-139.947784626047